https://huggingface.co/docs/transformers/tasks/question_answering

Installing required libraries

In [1]:
pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


Loading a subset of squad dataset

In [2]:
from datasets import load_dataset

squad_ds = load_dataset("squad", split = "train[:8000]")

squad_ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 8000
})

Splitting the dataset’s train split into a train and test set with the train_test_split method

In [3]:
squad_ds = squad_ds.train_test_split(test_size = 0.2)

squad_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 6400
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1600
    })
})

Looking at an instance

In [4]:
squad_ds["train"][5]

{'id': '56d28ce759d6e41400146093',
 'title': 'Buddhism',
 'context': 'According to a demographic analysis reported by Peter Harvey (2013): Mahayana has 360 million adherents; Theravada has 150 million adherents; and Vajrayana has 18,2 million adherents. Seven million additional Buddhists are found outside of Asia.',
 'question': 'How many adherents does Thervada have?',
 'answers': {'text': ['150 million'], 'answer_start': [119]}}

Now extracting context , question, and answers.The context and question fields are very straightforward to use. The answers field is a bit trickier as it comports a dictionary with two fields that are both lists. This is the format that will be expected by the squad metric during evaluation; if you are using your own data, you don’t necessarily need to worry about putting the answers in the same format. The text field is rather obvious, and the answer_start field contains the starting character index of each answer in the context.

In [5]:
squad_ds["train"][5]["context"]

'According to a demographic analysis reported by Peter Harvey (2013): Mahayana has 360 million adherents; Theravada has 150 million adherents; and Vajrayana has 18,2 million adherents. Seven million additional Buddhists are found outside of Asia.'

In [6]:
squad_ds["train"][5]["question"]

'How many adherents does Thervada have?'

Note that in this dataset we only have the answer start token position. In order to train the model we need the answer end as well - this is something we have to compute by adding the length of the answer to the start

In [7]:
squad_ds["train"][5]["answers"]

{'text': ['150 million'], 'answer_start': [119]}

In [8]:
squad_ds["train"][20]

{'id': '56daf04ae7c41114004b4b4b',
 'title': 'American_Idol',
 'context': 'The show had originally planned on having four judges following the Pop Idol format; however, only three judges had been found by the time of the audition round in the first season, namely Randy Jackson, Paula Abdul and Simon Cowell. A fourth judge, radio DJ Stryker, was originally chosen but he dropped out citing "image concerns". In the second season, New York radio personality Angie Martinez had been hired as a fourth judge but withdrew only after a few days of auditions due to not being comfortable with giving out criticism. The show decided to continue with the three judges format until season eight. All three original judges stayed on the judging panel for eight seasons.',
 'question': 'When did American Idol change to a four-judge panel?',
 'answers': {'text': ['season eight'], 'answer_start': [599]}}

In [9]:
squad_ds["train"][1000]

{'id': '56d5fa0e1c85041400946e86',
 'title': 'Dog',
 'context': "Coyotes and big cats have also been known to attack dogs. Leopards in particular are known to have a predilection for dogs, and have been recorded to kill and consume them regardless of the dog's size or ferocity. Tigers in Manchuria, Indochina, Indonesia, and Malaysia are reputed to kill dogs with the same vigor as leopards. Striped hyenas are major predators of village dogs in Turkmenistan, India, and the Caucasus. Reptiles such as alligators and pythons have been known to kill and eat dogs.",
 'question': 'What two reptiles kill dogs and consume them?',
 'answers': {'text': ['alligators and pythons'], 'answer_start': [438]}}

During training, there is only one possible answer. We can double-check this by using the Dataset.filter() method:

In [10]:
squad_ds["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/6400 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

We would be starting to preprocess train data... The hard part will be to generate labels for the question’s answer, which will be the start and end positions of the tokens corresponding to the answer inside the context.
 First, we  convert the text in the input into IDs the model can make sense of, using a tokenizer

In [11]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer.is_fast

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

True

We can pass to our tokenizer the question and the context together, and it will properly insert the special tokens to form a sentence.

If we see the input_ids, 101 is CLS token and 102 is SEP token

In [12]:
context = squad_ds["train"][3]["context"]
question = squad_ds["train"][3]["question"]

question, context

('Which library was built at Notre Dame in 1963?',
 "Since the construction of its oldest buildings, the university's physical plant has grown substantially. Over the years 29 residence halls have been built to accommodate students and each has been constructed with its own chapel. Many academic building were added together with a system of libraries, the most prominent of which is the Theodore Hesburgh Library, built in 1963 and today containing almost 4 million books. Since 2004, several buildings have been added, including the DeBartolo Performing Arts Center, the Guglielmino Complex, and the Jordan Hall of Science. Additionally, a new residence for men, Duncan Hall, was begun on March 8, 2007, and began accepting residents for the Fall 2008 semester. Ryan Hall was completed and began housing undergraduate women in the fall of 2009. A new engineering building, Stinson-Remick Hall, a new combination Center for Social Concerns/Institute for Church Life building, Geddes Hall, and a law

In [13]:
inputs = tokenizer(question, context)

inputs

{'input_ids': [101, 2029, 3075, 2001, 2328, 2012, 10289, 8214, 1999, 3699, 1029, 102, 2144, 1996, 2810, 1997, 2049, 4587, 3121, 1010, 1996, 2118, 1005, 1055, 3558, 3269, 2038, 4961, 12381, 1012, 2058, 1996, 2086, 2756, 5039, 9873, 2031, 2042, 2328, 2000, 8752, 2493, 1998, 2169, 2038, 2042, 3833, 2007, 2049, 2219, 4970, 1012, 2116, 3834, 2311, 2020, 2794, 2362, 2007, 1037, 2291, 1997, 8860, 1010, 1996, 2087, 4069, 1997, 2029, 2003, 1996, 10117, 2002, 9695, 2232, 3075, 1010, 2328, 1999, 3699, 1998, 2651, 4820, 2471, 1018, 2454, 2808, 1012, 2144, 2432, 1010, 2195, 3121, 2031, 2042, 2794, 1010, 2164, 1996, 2139, 8237, 3406, 4135, 4488, 2840, 2415, 1010, 1996, 19739, 25394, 2884, 10020, 2080, 3375, 1010, 1998, 1996, 5207, 2534, 1997, 2671, 1012, 5678, 1010, 1037, 2047, 5039, 2005, 2273, 1010, 7343, 2534, 1010, 2001, 5625, 2006, 2233, 1022, 1010, 2289, 1010, 1998, 2211, 10564, 3901, 2005, 1996, 2991, 2263, 13609, 1012, 4575, 2534, 2001, 2949, 1998, 2211, 3847, 8324, 2308, 1999, 1996, 2991, 1

In [14]:
tokenizer.decode(inputs["input_ids"])

"[CLS] which library was built at notre dame in 1963? [SEP] since the construction of its oldest buildings, the university's physical plant has grown substantially. over the years 29 residence halls have been built to accommodate students and each has been constructed with its own chapel. many academic building were added together with a system of libraries, the most prominent of which is the theodore hesburgh library, built in 1963 and today containing almost 4 million books. since 2004, several buildings have been added, including the debartolo performing arts center, the guglielmino complex, and the jordan hall of science. additionally, a new residence for men, duncan hall, was begun on march 8, 2007, and began accepting residents for the fall 2008 semester. ryan hall was completed and began housing undergraduate women in the fall of 2009. a new engineering building, stinson - remick hall, a new combination center for social concerns / institute for church life building, geddes hall

In [15]:
squad_ds["train"][3]["answers"]

{'text': ['Theodore Hesburgh Library'], 'answer_start': [336]}

Tokenizing only with truncation - this is a problem because the answer may not be present in the truncated context at all


In [16]:
inputs = tokenizer(
    question,
    context,
    max_length = 30,
    truncation = "only_second",
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS]
which
library
was
built
at
notre
dame
in
1963
?
[SEP]
since
the
construction
of
its
oldest
buildings
,
the
university
'
s
physical
plant
has
grown
substantially
[SEP]


Now we are tokenizing with some additional parameters arguments

- Only truncate the second input (the context)
- To also pass the truncated tokens to the model, you can use the return_overflowing_tokens (this will tokenize a single question-context into multiple question-context pairs i.e. several training features)
- Your model will probably face problems to generate meaningful embeddings (for your downstream task) for the tokens at the beginning and the end of each sentence since they are lacking context due to the hard-cut approach.The standard approach for that problem is the sliding window approach which keeps some tokens of the previous sequence for the current sequence. You can control the sliding window with the stride parameter of the tokenizer:

In [17]:
inputs = tokenizer(
    question,
    context,
    max_length = 30,
    truncation = "only_second",
    return_overflowing_tokens=True
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] which library was built at notre dame in 1963? [SEP] since the construction of its oldest buildings, the university's physical plant has grown substantially [SEP]
[CLS] which library was built at notre dame in 1963? [SEP]. over the years 29 residence halls have been built to accommodate students and each has been [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] constructed with its own chapel. many academic building were added together with a system of libraries [SEP]
[CLS] which library was built at notre dame in 1963? [SEP], the most prominent of which is the theodore hesburgh library, built in 1963 [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] and today containing almost 4 million books. since 2004, several buildings have been added, [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] including the debartolo performing arts center, the guglielmino complex [SEP]
[CLS] which library was built at notre dame in 1963? [SEP], and the j

In [18]:
inputs = tokenizer(
    question,
    context,
    max_length = 30,
    truncation = "only_second",
    stride = 5,
    return_overflowing_tokens=True
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] which library was built at notre dame in 1963? [SEP] since the construction of its oldest buildings, the university's physical plant has grown substantially [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] physical plant has grown substantially. over the years 29 residence halls have been built to accommodate [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] have been built to accommodate students and each has been constructed with its own chapel. many [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] its own chapel. many academic building were added together with a system of libraries, the [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] system of libraries, the most prominent of which is the theodore hesburgh library, [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] hesburgh library, built in 1963 and today containing almost 4 million books. since [SEP]
[CLS] which library was built at notre dame in 1

In this case the context is not too long, but some of the examples in the dataset have very long contexts that will exceed the maximum length we set (which is 384 in this case). when we explored the internals of the question-answering pipeline, we will deal with long contexts by creating several training features from one sample of our dataset, with a sliding window between them.

To see how this works using the current example, we can limit the length to 50 and use a sliding window of 20 tokens. As a reminder, we use:

- max_length to set the maximum length (here 50)
- truncation="only_second" to truncate the context (which is in the second position) when the question with its context is too long
- stride to set the number of overlapping tokens between two successive chunks (here 20)
- return_overflowing_tokens=True to let the tokenizer know we want the overflowing tokens.

As we can see, our example has been in split into 5 inputs, each of them containing the question and some part of the context. Note that the answer to the question only appears in the 1st and 2nd input, so by dealing with long contexts in this way we will create some training examples where the answer is not included in the context. For those examples, the labels will be start_position = end_position = 0 (so we predict the [CLS] token). We will also set those labels in the unfortunate case where the answer has been truncated so that we only have the start (or end) of it. For the examples where the answer is fully in the context, the labels will be the index of the token where the answer starts and the index of the token where the answer ends.

The dataset provides us with the start character of the answer in the context, and by adding the length of the answer, we can find the end character in the context. To map those to token indices, we will need to use the offset mappings. We can have our tokenizer return these by passing along return_offsets_mapping=True:

In [19]:
inputs = tokenizer(
    question,
    context,
    max_length = 50,
    truncation = "only_second",
    stride = 20,
    return_overflowing_tokens = True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] which library was built at notre dame in 1963? [SEP] since the construction of its oldest buildings, the university's physical plant has grown substantially. over the years 29 residence halls have been built to accommodate students and each has been constructed with its [SEP]
[CLS] which library was built at notre dame in 1963? [SEP]. over the years 29 residence halls have been built to accommodate students and each has been constructed with its own chapel. many academic building were added together with a system of libraries, the most [SEP]
[CLS] which library was built at notre dame in 1963? [SEP] constructed with its own chapel. many academic building were added together with a system of libraries, the most prominent of which is the theodore hesburgh library, built in 1963 and today containing [SEP]
[CLS] which library was built at notre dame in 1963? [SEP], the most prominent of which is the theodore hesburgh library, built in 1963 and today containing almost 4 million books.

In [20]:
inputs = tokenizer(
    question,
    context,
    max_length = 50,
    truncation = "only_second",
    stride = 20,
    return_overflowing_tokens = True,
    return_offsets_mapping = True,
)

inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [21]:
for i, offset in enumerate(inputs["offset_mapping"]):
  print(i, offset)

0 [(0, 0), (0, 5), (6, 13), (14, 17), (18, 23), (24, 26), (27, 32), (33, 37), (38, 40), (41, 45), (45, 46), (0, 0), (0, 5), (6, 9), (10, 22), (23, 25), (26, 29), (30, 36), (37, 46), (46, 47), (48, 51), (52, 62), (62, 63), (63, 64), (65, 73), (74, 79), (80, 83), (84, 89), (90, 103), (103, 104), (105, 109), (110, 113), (114, 119), (120, 122), (123, 132), (133, 138), (139, 143), (144, 148), (149, 154), (155, 157), (158, 169), (170, 178), (179, 182), (183, 187), (188, 191), (192, 196), (197, 208), (209, 213), (214, 217), (0, 0)]
1 [(0, 0), (0, 5), (6, 13), (14, 17), (18, 23), (24, 26), (27, 32), (33, 37), (38, 40), (41, 45), (45, 46), (0, 0), (103, 104), (105, 109), (110, 113), (114, 119), (120, 122), (123, 132), (133, 138), (139, 143), (144, 148), (149, 154), (155, 157), (158, 169), (170, 178), (179, 182), (183, 187), (188, 191), (192, 196), (197, 208), (209, 213), (214, 217), (218, 221), (222, 228), (228, 229), (230, 234), (235, 243), (244, 252), (253, 257), (258, 263), (264, 272), (273,

As we can see, we get back the usual input IDs, token type IDs, and attention mask, as well as the offset mapping we required and an extra key, overflow_to_sample_mapping. The corresponding value will be of use to us when we tokenize several texts at the same time (which we should do to benefit from the fact that our tokenizer is backed by Rust). Since one sample can give several features, it maps each feature to the example it originated from. Because here we only tokenized one example, we get a list of 0s:

In [22]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

But if we tokenize more examples, this will become more useful. Note that first 7 zeros are coming from above example

In [31]:
inputs = tokenizer(
    squad_ds["train"][4:6]["question"],
    squad_ds["train"][4:6]["context"],
    max_length = 50,
    truncation = "only_second",
    stride = 20,
    return_overflowing_tokens = True,
    return_offsets_mapping = True,
)

print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 3 examples gave 10 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1].


In [32]:
print(inputs.sequence_ids(0))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [33]:
len(inputs["offset_mapping"])

10

In [35]:
answers = squad_ds["train"][4:6]["answers"]

start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):

    print(i)
    # print(offset)

    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]

    print(answer)

    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])

    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    # Context tokens have sequence_id of 1
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

0
{'text': ['six'], 'answer_start': [324]}
1
{'text': ['six'], 'answer_start': [324]}
2
{'text': ['six'], 'answer_start': [324]}
3
{'text': ['six'], 'answer_start': [324]}
4
{'text': ['six'], 'answer_start': [324]}
5
{'text': ['six'], 'answer_start': [324]}
6
{'text': ['six'], 'answer_start': [324]}
7
{'text': ['six'], 'answer_start': [324]}
8
{'text': ['150 million'], 'answer_start': [119]}
9
{'text': ['150 million'], 'answer_start': [119]}


([0, 0, 0, 0, 39, 27, 0, 0, 36, 18], [0, 0, 0, 0, 39, 27, 0, 0, 37, 19])

### TODO Recording:

- Please look at the output above and then specify the index below
- IMPORTANT: The theoretical answer should match the actual label

Let’s take a look at a few results to verify that our approach is correct. For the first feature we find (44,45) as labels, so let’s compare the theoretical answer with the decoded span of tokens from 44 to 45 (inclusive):

In [ ]:
idx = 7
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]

labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: innovation, labels give: innovation


### TODO Recording:

- Please look at the output earlier and then specify the index below
- IMPORTANT: The theoretical answer should NOT match the actual label (label will be CLS)

In [ ]:
idx = 10
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]

labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: 36,620, labels give: [CLS]


There are a few preprocessing steps particular to question answering tasks you should be aware of:

Some examples in a dataset may have a very long context that exceeds the maximum input length of the model. To deal with longer sequences, truncate only the context by setting truncation="only_second".
Next, map the start and end positions of the answer to the original context by setting return_offset_mapping=True.
With the mapping in hand, now you can find the start and end tokens of the answer. Use the sequence_ids method to find which part of the offset corresponds to the question and which corresponds to the context.
Here is how you can create a function to truncate and map the start and end tokens of the answer to the context

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length = 384,
        truncation = "only_second",
        return_offsets_mapping = True,
        padding = "max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")

    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]

        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])

        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [ ]:
squad_ds["train"].column_names

['id', 'title', 'context', 'question', 'answers']

To apply the preprocessing function over the entire dataset, we use 🤗 Datasets map function. We can speed up the map function by setting batched=True to process multiple elements of the dataset at once. Remove any columns you don’t need.

context and question -> input_ids

answers -> start_positions, end_positions

In [ ]:
tokenized_squad = squad_ds.map(preprocess_function, batched = True, remove_columns = squad_ds["train"].column_names)

tokenized_squad

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 6400
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1600
    })
})

Now creating  a batch of examples using DefaultDataCollator. Unlike other data collators in 🤗 Transformers, the DefaultDataCollator does not apply any additional preprocessing such as padding.

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors = "tf")

For finetuning a model in TensorFlow, we start by setting up an optimizer function, learning rate schedule, and some training hyperparameters

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2

total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr = 2e-5,
    num_warmup_steps = 0,
    num_train_steps = total_train_steps,
)

We load  DistilBERT with TFAutoModelForQuestionAnswering:

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

Converting train and validation datasets to the tf.data.Dataset format with prepare_tf_dataset()

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle = False,
    batch_size = 16,
    collate_fn = data_collator,
)

Configuring the model for training with compile:

In [ ]:
import tensorflow as tf

model.compile(optimizer = optimizer)

model.summary()

Model: "tf_distil_bert_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 qa_outputs (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66364418 (253.16 MB)
Trainable params: 66364418 (253.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir = "QA_model_with_squad",
    tokenizer = tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/jinxxx123/QA_model_with_squad into local empty directory.


In [ ]:
model.fit(x = tf_train_set, validation_data = tf_validation_set, epochs = 3, callbacks = [callback])

Epoch 1/3
400/400 [==============================] - 319s 747ms/step - loss: 2.9370 - val_loss: 1.8826
Epoch 2/3
400/400 [==============================] - 309s 773ms/step - loss: 1.5361 - val_loss: 1.6364
Epoch 3/3
400/400 [==============================] - 291s 729ms/step - loss: 1.3296 - val_loss: 1.6364


### TODO Recording:

- Paste in one example at a time and run all the code cells

In [ ]:
question = "Who is credited with the discovery of gravity?"

context = "In the 20th century, Albert Einstein revolutionized our understanding of gravity with his theory of general relativity.\
           This theory describes gravity as the curvature of spacetime caused by mass and energy."

In [ ]:
#question = "How many chambers does the human heart have?"
#context = "The human heart is a muscular organ responsible for pumping blood throughout the body. \
#           It consists of four chambers: two atria and two ventricles. The left ventricle is responsible\
#           for pumping oxygenated blood to the body."

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model = model, tokenizer=tokenizer)
question_answerer(question = question, context = context)

{'score': 0.6052092909812927,
 'start': 21,
 'end': 36,
 'answer': 'Albert Einstein'}

Tokenizing the text and returning TensorFlow tensors

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("QA_model_with_squad")

inputs = tokenizer(question, context, return_tensors = "tf")

Passing inputs to the model and return the logits:

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("QA_model_with_squad")

outputs = model(**inputs)

Some layers from the model checkpoint at QA_model_with_squad were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at QA_model_with_squad and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Getting the highest probability from the model output for the start and end positions

In [ ]:
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

Answer is decoded from predicted tokens

In [ ]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'albert einstein'